In [1]:
import pandas as pd
df = pd.read_excel('../data/bronze/synthese_2013_with_filters_en.xls')
df

,Schengen State,Country where consulate is located,Consulate,A Visas applied for,A visas issued (including multiple),Multiple A visas issued,A visas not issued,Not issued rate for A visas,C visas applied for,Total C uniform visas issued (including MEV) \n,Multiple entry C uniform visas (MEVs) issued (for Germany: only MEVs valid for more than 1 year are included),Share of MEVs on total number of C uniform visas issued,Total C LTV visas issued,C visas not issued,Not issued rate for C visas,Total A and C visas applied for,"Total A and C visas issued (including multiple A visas, multiple-entry C visas and LTVs)",Total A and C visas not issued,Not issued rate for A and C visas
0,Finland,RUSSIAN FEDERATION,ST. PETERSBURG,1.0,1.0,NaN,NaN,0.000000,1204670.0,1195130.0,1171713.0,0.980406,35.0,9505.0,0.007890,1204671.0,1195166.0,9505.0,0.007890
1,Spain,RUSSIAN FEDERATION,MOSCOW,2.0,1.0,NaN,NaN,0.000000,1051643.0,1016986.0,196054.0,0.192779,3991.0,10292.0,0.009787,1051645.0,1020978.0,10292.0,0.009787
2,Greece,RUSSIAN FEDERATION,MOSCOW,1.0,1.0,NaN,NaN,0.000000,910824.0,908680.0,332319.0,0.365716,13.0,2131.0,0.002340,910825.0,908694.0,2131.0,0.002340
3,Italy,RUSSIAN FEDERATION,MOSCOW,0.0,0.0,0.0,0.0,NaN,725524.0,720863.0,707109.0,0.980920,270.0,4391.0,0.006052,725524.0,721133.0,4391.0,0.006052
4,France,RUSSIAN FEDERATION,MOSCOW,11.0,9.0,0.0,2.0,0.181818,390161.0,385160.0,117195.0,0.304276,275.0,4726.0,0.012113,390172.0,385444.0,4728.0,0.012118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,NaN,NaN,Selection Sub total in 2013,13253.0,11742.0,1979.0,1359.0,0.102543,17285449.0,16184173.0,7267987.0,0.449080,176948.0,831574.0,0.048108,17221508.0,16297790.0,831272.0,0.048269
1996,NaN,NaN,Total worldwide 2013,13253.0,11742.0,1979.0,1359.0,0.102543,17285449.0,16184173.0,7267987.0,0.449080,176948.0,831574.0,0.048108,17221508.0,16297790.0,831272.0,0.048269


Clean-up raw Schengen input file and save as silver statistics 

In [2]:
import schengentools.visastatsparser as vp
job = vp.Parser(2013)
job.extract('../data/bronze/synthese_2013_with_filters_en.xls', 0)
job.transform()
job.load()
df = job.df

Input file loaded into dataframe
Renamed columns
Trimmed country and city names
Removed irrelevant rows such as summary rows
Selected columns
Replaced applied, issued and not issued null values with 0
Replaced negative entries for visas issued with 0
Calculate refusal rate as the not issued share of the total issued and not issued
Added column with year issued and not issued


/Users/hobolthwagner/Source/schengen-visa/data-engineering/schengentools/visastatsparser.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["visas_issued"].loc[self.df["visas_issued"]<0] = 0


Data quality checks

In [3]:
df[df["visas_applied"] != df["visas_issued"] + df["visas_not_issued"]]

,schengen_state,origin_country,origin_consulate,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year
0,Finland,RUSSIAN FEDERATION,ST. PETERSBURG,1204670.0,1195130.0,9505.0,0.007890,2013
1,Spain,RUSSIAN FEDERATION,MOSCOW,1051643.0,1016986.0,10292.0,0.010019,2013
2,Greece,RUSSIAN FEDERATION,MOSCOW,910824.0,908680.0,2131.0,0.002340,2013
3,Italy,RUSSIAN FEDERATION,MOSCOW,725524.0,720863.0,4391.0,0.006054,2013
4,France,RUSSIAN FEDERATION,MOSCOW,390161.0,385160.0,4726.0,0.012121,2013
...,...,...,...,...,...,...,...,...
1950,Poland,ESTONIA,TALLINN,1.0,0.0,0.0,NaN,2013
1956,Portugal,GERMANY,HAMBURG,1.0,0.0,0.0,NaN,2013
1961,Spain,HUNGARY,BUDAPEST,1.0,0.0,0.0,NaN,2013
1962,Spain,NORWAY,OSLO,1.0,0.0,0.0,NaN,2013


In [4]:
df[df["visa_refusal_rate"]>1]

,schengen_state,origin_country,origin_consulate,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year


In [5]:
df[df["visa_refusal_rate"].isna()]

,schengen_state,origin_country,origin_consulate,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year
1329,Spain,SWITZERLAND,ZURICH,149.0,0.0,0.0,NaN,2013
1468,France,GERMANY,BERLIN,57.0,0.0,0.0,NaN,2013
1489,Sweden,SINGAPORE,SINGAPORE,51.0,0.0,0.0,NaN,2013
1594,Spain,GERMANY,MUNICH,25.0,0.0,0.0,NaN,2013
1693,Sweden,FRANCE,PARIS,14.0,0.0,0.0,NaN,2013
1696,Spain,AUSTRIA,VIENNA,13.0,0.0,0.0,NaN,2013
1723,France,BURUNDI,BUJUMBURA,10.0,0.0,0.0,NaN,2013
1730,Denmark,GERMANY,FLENSBURG,9.0,0.0,0.0,NaN,2013
1731,Denmark,HONG KONG S.A.R.,HONG KONG,9.0,0.0,0.0,NaN,2013
1759,Sweden,HONG KONG S.A.R.,HONG KONG,8.0,0.0,0.0,NaN,2013


In [6]:
df.describe()

,visas_applied,visas_issued,visas_not_issued,visa_refusal_rate,year
count,1.991000e+03,1.991000e+03,1991.000000,1953.000000,1991.0
mean,8.681793e+03,8.128752e+03,417.666499,0.102048,2013.0
std,4.925300e+04,4.837411e+04,1609.109628,0.170094,0.0
min,0.000000e+00,0.000000e+00,0.000000,0.000000,2013.0
25%,4.850000e+01,3.600000e+01,1.000000,0.002494,2013.0
50%,6.920000e+02,5.390000e+02,22.000000,0.033757,2013.0
75%,3.773500e+03,3.054500e+03,246.000000,0.123387,2013.0
max,1.204670e+06,1.195130e+06,41748.000000,1.000000,2013.0
